In [ ]:
!pip install -U transformers

## Local Inference on GPU
Model page: https://huggingface.co/google/medgemma-4b-it

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/medgemma-4b-it)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

In [ ]:
from transformers import pipeline

pipe = pipeline("image-text-to-text", model="google/medgemma-4b-it")
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    },
]
pipe(text=messages)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip "/content/drive/MyDrive/vlm files/archive.zip" -d "/content/covid_dataset"

## Remote Inference via Inference Providers
Ensure you have a valid **HF_TOKEN** set in your environment, running this may bill your account above the free tier.
The following Python example shows how to run the model remotely on HF Inference Providers, using the **auto** provider setting (automatically selects an available inference provider).

In [ ]:
from transformers import pipeline
from PIL import Image

pipe = pipeline("image-text-to-text", model="google/medgemma-4b-it", device_map="auto")

image_paths = image_files[:20]
results = []

for path in image_paths:
    image = Image.open(path).convert("RGB")

    messages = [{
        "role": "user",
        "content": [
            {"type": "image", "image": image},
            {"type": "text", "text": "Does this chest x-ray display signs of pneumonia?"}
        ]
    }]

    try:
        output = pipe(text=messages)
        response = output[0]["generated_text"][-1]["content"]
    except Exception as e:
        response = f"ERROR: {str(e)}"

    results.append((os.path.basename(path), response))

# Save CSV
import pandas as pd
df = pd.DataFrame(results, columns=["Image", "Prediction"])
df.to_csv("covid_inference_20.csv", index=False)

from google.colab import files
files.download("covid_inference_20.csv")

In [ ]:
from transformers import pipeline
from PIL import Image
import pandas as pd
from tqdm import tqdm
import os

#working version for all files

pipe = pipeline("image-text-to-text", model="google/medgemma-4b-it", device_map="auto")
output_path = "covid_inference_all.csv"

# loads results
if os.path.exists(output_path):
    df_existing = pd.read_csv(output_path)
    processed_files = set(df_existing["Image"])
    results = df_existing.values.tolist()
    print(f"Resuming from {len(results)} already processed images.")
else:
    processed_files = set()
    results = []

# runs inference
for path in tqdm(image_files, desc="Processing images"):
    filename = os.path.basename(path)
    if filename in processed_files:
        continue

    try:
        image = Image.open(path).convert("RGB")
        messages = [{
            "role": "user",
            "content": [
                {"type": "image", "image": image},
                {"type": "text", "text": "Does this chest x-ray display signs of pneumonia?"}
            ]
        }]
        output = pipe(text=messages)
        response = output[0]["generated_text"][-1]["content"]
    except Exception as e:
        response = f"ERROR: {str(e)}"

    results.append((filename, response))
    pd.DataFrame(results, columns=["Image", "Prediction"]).to_csv(output_path, index=False)

from google.colab import files
files.download(output_path)
